In [6]:
import config
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy
from sqlalchemy import MetaData
import requests
import missingno as msno
from matplotlib import rc_context
from matplotlib import ticker as tick
import matplotlib.pyplot as plt
import matplotlib.dates as md
%matplotlib inline
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
from scipy import stats
import datetime as dt
import time
from datetime import date
sns.set(style="darkgrid")
import warnings
warnings.simplefilter("ignore")

In [10]:
API_KEY = config.API_KEY
CHANNEL_ID = 'UC0WNX0de8I1fgKUrjxiPaRA'
#tutorial UCW8Ews7tdKKkBT6GdtQaXvQ
#BlindRun UC0WNX0de8I1fgKUrjxiPaRA
S_DATE = '2017-04-19'
E_DATE = '2022-04-19'
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)
ENG_CONFIG = f"mysql://{config.DB_USER}:{config.DB_PASS}@{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}"


In [53]:
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])

In [58]:
    def get_stats(video_id): 
    
        url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        
        return view_count, like_count, comment_count

In [126]:
def get_vids(df):
    #make an api call
    pageToken = "CDIQAA"
    #nextPageToken = 'CC0QAA'
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=100&"+pageToken
    response = requests.get(url).json()
    print(response)
    time.sleep(1)

    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            #video_title = str(video_title).replace("&amp;","") format string as necessary
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]

            view_count, like_count, comment_count = get_stats(video_id)

        df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,
                        'view_count':view_count,'like_count':like_count,'comment_count':comment_count}
                        , ignore_index = True)
    return(df)

In [127]:
1586

1586

In [128]:
df = get_vids(df)



{'kind': 'youtube#searchListResponse', 'etag': 'CS2wFThJ6glYlOdpvIDTq1O4soI', 'nextPageToken': 'CDIQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 1586, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'cFYF1GLGGf4J0--YeedGQiQUWSQ', 'id': {'kind': 'youtube#video', 'videoId': 'j7clspfC-_o'}, 'snippet': {'publishedAt': '2022-04-21T23:00:28Z', 'channelId': 'UC0WNX0de8I1fgKUrjxiPaRA', 'title': 'ADA Lovelace Memory Faster and More Efficient', 'description': 'NVIDIA Allegedly Begins Testing Its Fastest Next-Gen GPU, The AD102, For GeForce RTX 4090 Graphics Card, Features 24 ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/j7clspfC-_o/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/j7clspfC-_o/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/j7clspfC-_o/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Son of a Tech', 'liveBroadcastContent': 'none', 'publishTi

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       657 non-null    object
 1   video_title    657 non-null    object
 2   upload_date    657 non-null    object
 3   view_count     657 non-null    object
 4   like_count     657 non-null    object
 5   comment_count  657 non-null    object
dtypes: object(6)
memory usage: 30.9+ KB


In [117]:
df.head(30)

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,MZaoRfFgSRU,Intel Arc A-Series Performance Leak,2022-04-06,3687,155,27
1,4OBGxFA5onY,#99 - Binance Are Nazis?! KAPPA,2022-04-21,1690,183,6
2,MMNoGaYyoyA,#98 - Bitcoin Breaks $42000 on 4/20 | Intel Ar...,2022-04-20,3411,271,92
3,WKhkVx28Pgo,Crypto Exchange Balances are Liquidated,2022-04-20,4995,206,128
4,OwLk7lCTK3I,Is it Too Late To Start Mining Crypto,2022-04-19,7621,338,147
5,RvHTENTgakI,GPU Prices Falling,2022-04-19,2070,88,45
6,JnZE9z2pDLg,#97 - RX 6650 XT Hashrate Prediction | Monero ...,2022-04-19,4108,363,35
7,OiLxtdC9R9Y,RTX 3090TI Becomes Efficiency King,2022-04-19,8438,298,72
8,-S3PdvahOyY,TSMC Removes Crypto Mining From Earnings Report,2022-04-18,1833,88,62
9,3PZGKpXipdA,DOGE Pumps When Elon Does Anything,2022-04-18,1355,69,18


In [7]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host= host_name, dbname= dbname, user=username, password= password, port=port)

    except ps.OperationalError as e:
        raise e

    else:
        print('Connected!')
        
        return conn

In [8]:
#create the table
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL
                            )""")
    curr.execute(create_table_command)

In [9]:
def check_if_video_exists(curr, video_id):
    query= ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,))
    
    return curr.fetchone() is not None

In [10]:
def update_row(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(query, vars_to_update)

In [11]:
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])

    for i, row in df.iterrows():
        if check_if_video_exists(curr, video_id):
            update_row(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])
        else:
            tmp_df = tmp_df.append(row)
            
    return tmp_df

In [12]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, view_count, like_count, comment_count)
                         VALUES(%s, %s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [13]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [14]:

#conn = None

In [15]:
conn = connect_to_db(config.DB_HOST, config.DB_NAME, config.DB_PORT, config.DB_USER, config.DB_PASS)

Connected!


In [16]:
curr = conn.cursor()

In [17]:
create_table(curr)

In [18]:
new_vid_df = update_db(curr,df)

In [19]:
append_from_df_to_db(curr,new_vid_df)

In [20]:
conn.commit()

In [21]:
df.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,yFyyP_Uvxj0,How To Solve Data Science Probability Intervie...,2022-03-10,1113,44,2
1,Av92CaPTRr8,How to use SQL window functions: RANK vs DENSE...,2022-03-02,1624,50,8
2,VEjxlKBkZGM,Amazon Data Science Interview Question Walkthr...,2022-02-08,5146,166,16
3,hMUf7DqG1nQ,SQL Basics: How to Join Multiple Tables in SQL...,2022-01-27,2941,87,5
4,ToEl_tpyoM4,Uber Data Science Python Interview Question Wa...,2022-01-10,17345,440,24
